In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import os
import time

In [2]:
@jit
def update_forces(N, r, L, R, e, f):
    F = np.zeros((N, 3))
    F_s = 0
    for i in range(N):
        F_tmp = np.zeros(3)
        r_i = np.sqrt(r[i].dot(r[i]))
        if  r_i >= L:
            F_s_tmp = f*(L-r_i) / r_i * r[i]
            F[i] += F_s_tmp
            F_s += np.sqrt(np.dot(F_s_tmp,F_s_tmp))

    for i in range(N):
        for j in range(i):
            F_tmp = np.zeros(3)
            r_ij_vec = r[i] - r[j]
            r_ij = np.sqrt(r_ij_vec.dot(r_ij_vec))
            Rr = R / r_ij
            R6 = Rr*Rr*Rr*Rr*Rr*Rr
            R12 = R6*R6
            F_tmp = 12 * e * (R12 - R6) / (r_ij*r_ij)  * r_ij_vec
            F[i] += F_tmp
            F[j] -= F_tmp
    return F, F_s

In [3]:
def symulacja():
    # Parametry symulacji

    filename = "A.txt"
    data = {}

    file = open(filename)
    for line in file:
        data[line.split()[0]]= float(line.split()[1])
    file.close() 
    print(data)

    a = data['a']
    n = int(data['n'])
    T_0 = data['T_0']
    m = data['m']
    L = data['L']
    L = 1.4 * a * (n-1) ####################################
    f = data['f']
    e = data['e']
    R = data['R']
    tau = data['tau']
    S_0 = int(data['S_0'])
    S_d = int(data['S_d'])
    S_out = int(data['S_out'])
    S_xyz = int(data['S_xyz'])

    # Stan układu i warunki poczatkowe


    b0 = np.array([a, 0, 0])
    b1 = np.array([a/2, a*np.sqrt(3)/2, 0])
    b2 = np.array([a/2, a*np.sqrt(3)/6, a*np.sqrt(2/3)])

    N = n**3

    r = []

    for i0 in range(n):
        for i1 in range(n):
            for i2 in range(n):
                r.append((i0-(n-1)/2) * b0 + (i1-(n-1)/2) * b1 + (i2-(n-1)/2) * b2)

    k = 8.31e-3
    E_0 = []

    for _ in range(N):
        E_0.append([-1/2*k*T_0*np.log(np.random.uniform()), -1/2*k*T_0*np.log(np.random.uniform()), -1/2*k*T_0*np.log(np.random.uniform()),])

    p = []
    P = np.zeros(3)

    for i in range(N):
        p_tmp = np.array([np.sign(np.random.uniform(-1,1))*np.sqrt(2*m*E_0[i][0]), np.sign(np.random.uniform(-1,1))*np.sqrt(2*m*E_0[i][0]), np.sign(np.random.uniform(-1,1))*np.sqrt(2*m*E_0[i][0])])
        p.append(p_tmp)
        P = P + p_tmp

    P = P/N

    for i in range(N):
        p[i] = p[i] - P

    p = np.array(p)

    # fig, ax = plt.subplots(1,3)

    # ax[0].hist(p[:,0], bins=60)
    # ax[1].hist(p[:,1], bins=60)
    # ax[2].hist(p[:,2], bins=60)

    # plt.show()

    # Potencjały

    E = 0

    for i in range(N):
        E += p[i].dot(p[i])/(2*m)
        r_i = np.sqrt(r[i].dot(r[i]))
        if  r_i >= L:
            E += f/2*(L-r_i)*(L-r_i)

        for j in range(i):
                r_ij_vec = r[i] - r[j]
                r_ij = np.sqrt(r_ij_vec.dot(r_ij_vec))
                Rr = R / r_ij
                R6 = Rr*Rr*Rr*Rr*Rr*Rr
                R12 = R6*R6
                E += e*(R12 - 2*R6)
    # print(f'H = {E}')

    # Siły

    F = np.zeros((N, 3))

    for i in range(N):
        F_tmp = np.zeros(3)
        r_i = np.sqrt(r[i].dot(r[i]))
        if  r_i >= L:
            print("yes")
            F[i] += f*(L-r_i) / r_i * r[i]

    for i in range(N):
        for j in range(i):
            F_tmp = np.zeros(3)
            r_ij_vec = r[i] - r[j]
            r_ij = np.sqrt(r_ij_vec.dot(r_ij_vec))
            Rr = R / r_ij
            R6 = Rr*Rr*Rr*Rr*Rr*Rr
            R12 = R6*R6
            F_tmp += 12 * e * (R12 - R6) / (r_ij*r_ij)  * r_ij_vec
            F[i] += F_tmp
            F[j] -= F_tmp

    ##################### SYMULACJA ##############################

    # Równania ruchu

    T_av = 0
    P_av = 0
    H_av = 0
    
    prefix = f'Ar_n-{n}_T-{T_0}'
    
    current_directory = os.getcwd()
    final_directory = os.path.join(current_directory, prefix)
    if not os.path.exists(final_directory):
        os.makedirs(final_directory)
    
    f1 = open("./" + prefix + "/" + prefix + "_jmol.txt", "w+")
    f3 = open("./" + prefix + "/" + prefix + ".txt", "w+")
    f2 = open("./" + prefix + "/" + prefix + ".out", "w+")
    
    start = time.time()
    for s in range(S_0 + S_d):

        p_tmp = p + F*tau/2

        r = r + p_tmp*tau/m

        F, F_s = update_forces(N,r,L,R,e,f)

        p = p_tmp + F*tau/2

        if s % S_out == 0:
            E = 0
            E_kin = 0
            for i in range(N):
                E_kin += p[i].dot(p[i])/(2*m)

                r_i = np.sqrt(r[i].dot(r[i]))
                if  r_i >= L:
                    E += f/2*(L-r_i)*(L-r_i)

                for j in range(i):
                    r_ij_vec = r[i] - r[j]
                    r_ij = np.sqrt(r_ij_vec.dot(r_ij_vec))
                    Rr = R / r_ij
                    R6 = Rr*Rr*Rr*Rr*Rr*Rr
                    R12 = R6*R6
                    E += e*(R12 - 2*R6)
            H = E + E_kin
            P = F_s / (4*np.pi*L*L)
            T = 2/(3*N*k) * E_kin
            f2.write(f'{s} {H} {E} {E_kin} {T} {P}\n') # t H V T P

        if s % S_xyz == 0:
            E = 0
            E_kin = 0
            for i in range(N):
                E_kin += p[i].dot(p[i])/(2*m)

                r_i = np.sqrt(r[i].dot(r[i]))
                if  r_i >= L:
                    E += f/2*(L-r_i)*(L-r_i)

                for j in range(i):
                    r_ij_vec = r[i] - r[j]
                    r_ij = np.sqrt(r_ij_vec.dot(r_ij_vec))
                    Rr = R / r_ij
                    R6 = Rr*Rr*Rr*Rr*Rr*Rr
                    R12 = R6*R6
                    E += e*(R12 - 2*R6)
            H = E + E_kin
            # print(f'#{s} / {S_0 + S_d}\t H = {H}')
            f1.write(f'{N}\n\n')
            f3.write('ITEM: TIMESTEP\n')
            f3.write(str(s) + '\n')
            f3.write('ITEM: NUMBER OF ATOMS \n')
            f3.write(str(N) + '\n')
            f3.write('ITEM: ATOMS id type x y z \n')
            for i in range(N):
                f1.write(f'Ar {r[i][0]} {r[i][1]} {r[i][2]}\n')
                f3.write(f'{i} 1 {r[i][0]} {r[i][1]} {r[i][2]}\n')
            f1.write('\n')
    end = time.time()
    f1.close() 
    f3.close()
    f2.close()
    print(f'### T = {T_0}, n = {n}: elapsed time = {end-start}')

In [8]:
symulacja()

{'n': 4.0, 'm': 40.0, 'e': 1.0, 'R': 0.38, 'f': 10000.0, 'L': 2.3, 'a': 0.38, 'T_0': 1000.0, 'tau': 0.001, 'S_0': 0.0, 'S_d': 10000.0, 'S_out': 10.0, 'S_xyz': 100.0}
### T = 1000.0, n = 4: elapsed time = 11.854372262954712
